In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np 
import time
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFE
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
#importing the needed library

In [2]:
def selectkbest(indep_x,dep_y,n):
    test=SelectKBest(score_func=chi2,k=n)
    fit1=test.fit(indep_x,dep_y)
    selectk_features=fit1.transform(indep_x)
    selected_indices = fit1.get_support(indices=True)
    selected_feature_names = indep_x.columns[selected_indices] 
    return selectk_features, selected_feature_names.tolist()  


In [3]:
#we integrated training and test set with standardscalar
def split_scalar(indep_x,dep_y):
    x_train,x_test,y_train,y_test=train_test_split(indep_x,dep_y,test_size=0.25,random_state=0)
    sc=StandardScaler()
    x_train=sc.fit_transform(x_train)
    x_test=sc.transform(x_test)
  
    return x_train, x_test, y_train, y_test, sc                                                                  


In [4]:
#we make confusion matrix as a function
def cm_prediction(classifier,x_test):
    y_pred=classifier.predict(x_test)
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test,y_pred)
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    accuracy=accuracy_score(y_test,y_pred)
    report=classification_report(y_test,y_pred)
    return classifier,accuracy, report,x_test,y_test,cm

In [5]:
def Decision(x_train,y_train,x_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier= DecisionTreeClassifier(criterion='entropy',random_state=0)
    classifier.fit(x_train, y_train.ravel())
    classifier,accuracy, report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return classifier,accuracy, report,x_test,y_test,cm


In [6]:
#table Formation
def selectK_classification(accdes):
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['Decision'])
    dataframe['Decision']=accdes
    return dataframe

In [7]:
dataset1=pd.read_csv('prep.csv',index_col=None)

In [8]:
df2=dataset1
df2

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [9]:
df2=pd.get_dummies(df2,drop_first=True)
indep_x=df2.drop('classification_yes',1)
dep_y=df2['classification_yes']
df2

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [10]:
kbest,selected_feature_names=selectkbest(indep_x,dep_y,5)

In [11]:
accdes=[]

In [12]:
print("Selected features:", selected_feature_names)

Selected features: ['bgr', 'bu', 'sc', 'pcv', 'wc']


In [13]:
kbest

array([[1.48112676e+02, 5.74821053e+01, 3.07735602e+00, 3.88689024e+01,
        8.40819113e+03],
       [1.48112676e+02, 2.20000000e+01, 7.00000000e-01, 3.40000000e+01,
        1.23000000e+04],
       [9.90000000e+01, 2.30000000e+01, 6.00000000e-01, 3.40000000e+01,
        8.40819113e+03],
       ...,
       [1.10000000e+02, 1.15000000e+02, 6.00000000e+00, 2.60000000e+01,
        9.20000000e+03],
       [2.07000000e+02, 8.00000000e+01, 6.80000000e+00, 3.88689024e+01,
        8.40819113e+03],
       [1.00000000e+02, 4.90000000e+01, 1.00000000e+00, 5.30000000e+01,
        8.50000000e+03]])

In [14]:
x_train,x_test,y_train,y_test,sc=split_scalar(kbest,dep_y)

In [15]:
classifier,accuracy, report,x_test,y_test,cm=Decision(x_train,y_train,x_test)
accdes.append(accuracy)
result=selectK_classification(accdes)

In [16]:
result

,Decision
ChiSquare,0.96


In [17]:
import pickle as pkl
filename="finalized_model_Decision.sav"

In [18]:
pickle.dump(classifier,open(filename,'wb'))

In [19]:
filenamex="sc.pkl"

In [20]:
pickle.dump(sc,open(filenamex,'wb'))

In [21]:
sc=pickle.load(open("sc.pkl",'rb'))

In [22]:
preinput = sc.transform([[148.112676, 57.482105, 3.077356, 38.868902, 8408.191126]])
preinput

array([[-0.00249481, -0.05488798, -0.02438061, -0.00680432,  0.06186409]])

In [23]:

loaded_model=pickle.load(open("finalized_model_Decision.sav",'rb')) 
result = loaded_model.predict(preinput)

In [24]:
result

array([1], dtype=uint8)